<a href="https://colab.research.google.com/github/xaldoxxx/traderman/blob/main/stoploss_takeprofitApalancamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
!pip install ccxt

In [ ]:
import yfinance as yf
import talib as ta
import plotly.graph_objects as go
from datetime import datetime, timedelta

class EstrategiaStopLossTakeProfit:
    def __init__(self, simbolo, fecha_inicio, fecha_fin, intervalo, dias, stop_loss, take_profit, apalancamiento):
        """
        Inicializa la clase con los parámetros necesarios.
        """
        self.simbolo = simbolo
        self.fecha_inicio = fecha_inicio
        self.fecha_fin = fecha_fin
        self.intervalo = intervalo
        self.dias = dias
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.apalancamiento = apalancamiento

    def obtener_datos_historicos(self):
        """
        Obtiene los datos históricos del símbolo especificado.
        """
        datos = yf.download(self.simbolo, start=self.fecha_inicio, end=self.fecha_fin, interval=self.intervalo)
        return datos

    def calcular_stop_loss_take_profit(self, datos):
        """
        Calcula los niveles de Stop Loss y Take Profit utilizando talib.
        """
        ultimo_precio = datos['Close'].iloc[-1]
        atr = ta.ATR(datos['High'], datos['Low'], datos['Close'], timeperiod=14).iloc[-1]
        stop_loss_price = ultimo_precio - (atr * self.stop_loss)
        take_profit_price = ultimo_precio + (atr * self.take_profit)
        return stop_loss_price, take_profit_price

    def calcular_apalancamiento(self, datos):
        """
        Calcula el apalancamiento basado en los datos históricos.
        """
        ultimo_precio = datos['Close'].iloc[-1]
        margen_requerido = ultimo_precio / self.apalancamiento
        return margen_requerido

    def tomar_decision(self, datos, stop_loss_price, take_profit_price, margen_requerido):
        """
        Toma una decisión basada en los niveles calculados y el apalancamiento.
        """
        ultimo_precio = datos['Close'].iloc[-1]
        if ultimo_precio <= stop_loss_price:
            return "Vender"
        elif ultimo_precio >= take_profit_price:
            return "Comprar"
        elif margen_requerido <= self.stop_loss:
            return "Esperar, Pérdida máxima alcanzada"
        elif margen_requerido >= self.apalancamiento:
            return "Esperar, Margen requerido excede el apalancamiento"
        else:
            return "Esperar"

    def graficar_datos(self, datos, stop_loss_price, take_profit_price):
        """
        Grafica los datos históricos junto con los niveles de Stop Loss y Take Profit.
        """
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=datos.index, y=datos['Close'], mode='lines', name='Precio de cierre', line=dict(color='black')))
        fig.add_trace(go.Scatter(x=datos.index, y=[stop_loss_price] * len(datos.index), mode='lines', name='Stop Loss', line=dict(color='red', dash='dash')))
        fig.add_trace(go.Scatter(x=datos.index, y=[take_profit_price] * len(datos.index), mode='lines', name='Take Profit', line=dict(color='green', dash='dash')))

        fig.update_layout(title='Estrategia de Stop Loss y Take Profit',
                          xaxis_title='Fecha',
                          yaxis_title='Precio')

        fig.show()


# Configuración de parámetros
simbolo = "DOGE-USD"
dias = 5
intervalo = "1m"
stop_loss = 0.05  # 5% de pérdida máxima permitida
take_profit = 0.1  # 10% de ganancia objetivo
apalancamiento = 3  # Apalancamiento de 10x

fecha_inicio = datetime.now() - timedelta(days=dias)
fecha_fin = datetime.now()

# Crear instancia de la clase EstrategiaStopLossTakeProfit
estrategia = EstrategiaStopLossTakeProfit(simbolo, fecha_inicio, fecha_fin, intervalo, dias, stop_loss, take_profit, apalancamiento)

# Obtener datos históricos
datos_historicos = estrategia.obtener_datos_historicos()

# Calcular Stop Loss y Take Profit
stop_loss_price, take_profit_price = estrategia.calcular_stop_loss_take_profit(datos_historicos)

# Calcular apalancamiento
margen_requerido = estrategia.calcular_apalancamiento(datos_historicos)

# Tomar decisión basada en la estrategia
decision = estrategia.tomar_decision(datos_historicos, stop_loss_price, take_profit_price, margen_requerido)
print("Decisión:", decision)

# Graficar datos
estrategia.graficar_datos(datos_historicos, stop_loss_price, take_profit_price)

In [ ]:
import yfinance as yf
import talib as ta
import plotly.graph_objects as go
from datetime import datetime, timedelta

class EstrategiaStopLossTakeProfit:
    def __init__(self, simbolo, fecha_inicio, fecha_fin, intervalo, dias, stop_loss, take_profit, apalancamiento):
        """
        Inicializa la clase con los parámetros necesarios.
        """
        self.simbolo = simbolo
        self.fecha_inicio = fecha_inicio
        self.fecha_fin = fecha_fin
        self.intervalo = intervalo
        self.dias = dias
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.apalancamiento = apalancamiento

    def obtener_datos_historicos(self):
        """
        Obtiene los datos históricos del símbolo especificado.
        """
        datos = yf.download(self.simbolo, start=self.fecha_inicio, end=self.fecha_fin, interval=self.intervalo)
        return datos

    def calcular_stop_loss_take_profit(self, datos):
        """
        Calcula los niveles de Stop Loss y Take Profit utilizando talib.
        """
        atr = ta.ATR(datos['High'], datos['Low'], datos['Close'], timeperiod=14).iloc[-1]
        ultimo_precio = datos['Close'].iloc[-1]
        stop_loss_price = ultimo_precio - (atr * self.stop_loss)
        take_profit_price = ultimo_precio + (atr * self.take_profit)
        return stop_loss_price, take_profit_price

    def calcular_apalancamiento(self, datos):
        """
        Calcula el apalancamiento basado en los datos históricos.
        """
        ultimo_precio = datos['Close'].iloc[-1]
        margen_requerido = ultimo_precio / self.apalancamiento
        return margen_requerido

    def tomar_decision(self, datos, stop_loss_price, take_profit_price, margen_requerido):
        """
        Toma una decisión basada en los niveles calculados y el apalancamiento.
        """
        ultimo_precio = datos['Close'].iloc[-1]
        if ultimo_precio <= stop_loss_price:
            return "Vender"
        elif ultimo_precio >= take_profit_price:
            return "Comprar"
        elif margen_requerido >= 1 / self.apalancamiento:
            return "Esperar, Margen requerido excede el apalancamiento"
        else:
            return "Esperar"

    def graficar_datos(self, datos, stop_loss_price, take_profit_price):
        """
        Grafica los datos históricos junto con los niveles de Stop Loss y Take Profit.
        """
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=datos.index, y=datos['Close'], mode='lines', name='Precio de cierre', line=dict(color='black')))
        fig.add_trace(go.Scatter(x=datos.index, y=[stop_loss_price] * len(datos.index), mode='lines', name='Stop Loss', line=dict(color='red', dash='dash')))
        fig.add_trace(go.Scatter(x=datos.index, y=[take_profit_price] * len(datos.index), mode='lines', name='Take Profit', line=dict(color='green', dash='dash')))

        fig.update_layout(title='Estrategia de Stop Loss y Take Profit',
                          xaxis_title='Fecha',
                          yaxis_title='Precio')

        fig.show()


# Configuración de parámetros
simbolo = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "ADA-USD", "SOL-USD", "DOT-USD", "USDT-USD", "DAI-USD", "USDC-USD", "LUNA-USD", "MATIC-USD", "AVAX-USD", "BNB-USD", "MANA-USD", "SAND-USD", "SLP-USD", "AXS-USD", "UNI-USD", "ALGO-USD", "AAVE-USD", "ENS-USD", "NEAR-USD", "FTM-USD", "PAXG-USD", "DOGE-USD", "TRX-USD", "ARB-USD", "LTC-USD", "XLM-USD", "OP-USD", "TRX-USD", "LUNA-USD"]
dias = "7"  #@param ["59", "45", "30", "20", "15", "7", "6", "5", "4", "3", "2", "1"]
intervalo = "5m" #@param ["1m", "5m", "15m", "60m", "90m", "120m", "1d"]

fecha_inicio = datetime.now() - timedelta(days=int(dias))
fecha_fin = datetime.now()

stop_loss = 0.05  # 5% de pérdida máxima permitida
take_profit = 0.1  # 10% de ganancia objetivo
apalancamiento = 3  # Apalancamiento de 3x


# Crear instancia de la clase EstrategiaStopLossTakeProfit
estrategia = EstrategiaStopLossTakeProfit(simbolo, fecha_inicio, fecha_fin, intervalo, dias, stop_loss, take_profit, apalancamiento)

# Obtener datos históricos
datos_historicos = estrategia.obtener_datos_historicos()

# Calcular Stop Loss y Take Profit
stop_loss_price, take_profit_price = estrategia.calcular_stop_loss_take_profit(datos_historicos)

# Calcular apalancamiento
margen_requerido = estrategia.calcular_apalancamiento(datos_historicos)

# Tomar decisión basada en la estrategia
decision = estrategia.tomar_decision(datos_historicos, stop_loss_price, take_profit_price, margen_requerido)
print("Decisión:", decision)

# Graficar datos
estrategia.graficar_datos(datos_historicos, stop_loss_price, take_profit_price)
